In [16]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import ode
from scipy.stats import * 

In [17]:
#Run this cell to create the Monte Carlo Pi Estimation Model
def PiEstimate(N):
    # N: Number of Monte Carlo samples
    # Setting seed to make the results reproducible, if not the random sample generation will change everytime 
    np.random.seed(42)
    # we execute this piece of code 
    f = np.zeros(N)
    # Drawing uniformly distributed random samples within the domain [-1x1]^2. "np.random.rand(2,1)" produces the samples 
    # only in the 1st quadrant but our domain extends in all 4 quadrants. The value -1 summed to the samples will shift 
    # the values of both the coordinates to negative values but when the factor 2 multipled to the samples will yield 
    # samples that spans the entire required domain.
    zi = -1 + 2*np.random.rand(N,2)
    z_in,z_out=[],[]
    plt.figure(figsize = [8,8])
    # Integrand-Indicator function
    for i,val in enumerate(zi):    
        
    # Computation of the radius of the sample 
      ri = np.sqrt(val[0]**2 + val[1]**2)    
    
    # Checking if the sample drawn is within the unit circle. If yes, f[i] takes the value of 1 else 0.
      f[i] = np.int(ri <= 1) 
      if f[i] == 1:
         z_in.append(val)
      else:
         z_out.append(val)
    
    z_in,z_out = np.array(z_in),np.array(z_out)     
    plt.scatter(x=z_in[:,0],y=z_in[:,1],color = 'r',s = 1)
    plt.scatter(x=z_out[:,0],y=z_out[:,1],color = 'b',s = 1)
    plt.show()
    # Monte Carlo approximation of pi - the factor 4 here is to consider all the 4 quadrants of the circle.
    pi_MC = 4*np.sum(f)/N   
    # Computing the error between true value and MC approximation
    print("Monte Carlo error %g" % (np.abs(pi_MC - np.pi)))

In [7]:

def Convergence(N1): 
  np.random.seed(42)
  Nvalues=[int(N1*0.25),int(N1*0.50),int(N1*0.75),N1]
  n = len(Nvalues)

# Creating an array to store the values of means and std. dev during the evaluation in the loop.
  mu_MC = np.zeros(n)
  sig_MC = np.zeros(n)

# Looping over the number of N values
  for j in range(n):
    
    # Converting the N values from float to integer datatype
    N = np.int(Nvalues[j])
    
    # Integrand-Indicator function
    f = np.zeros(N)
    
    # Looping over the corresponding N values
    for i in range(N):    
        zi = -1 + 2*np.random.rand(2,1)  
        ri = np.sqrt(zi[0,0]**2 + zi[1,0]**2)   
        f[i] = np.int(ri <= 1)  
    mu_MC[j] = np.mean(f)   
    sig_MC[j] = np.std(f)
    
  return mu_MC,sig_MC, Nvalues

In [19]:
def PlotConvergence(mu_MC,sig_MC, Nvalues):
   
    MC_errors = 5*np.divide(sig_MC,np.sqrt(Nvalues))
    fig=plt.figure(figsize=(12,8))
# Since there exist some uncertainty in the value of pi approximated by MC method, the confidence levels are given in the
# form of error bands around the approximated value represented by the green lines. 
    plt.errorbar(Nvalues, 4*mu_MC, yerr=MC_errors, label = 'convergence',ecolor = 'g', fmt='-o', capsize = 5, capthick = 2)

# The true value is given by the red line
    plt.plot(Nvalues,np.pi*np.ones(len(Nvalues)),'r-')
 
# X and Y labels of the graph
    plt.ylabel('Value of Pi')
    plt.xlabel('N values')
    plt.xscale('log')
    plt.title('Convergence Study')
    plt.show()

In [20]:
# Defining the model for the harmonic oscillator with the input parameters as the components of the 
# random vector Z.
def Q(z):
    gamma = z[0]    
    omega = z[1]
    f     = z[2]
    k     = z[3]
    x0    = z[4]
    x1    = z[5]

    # We need to define the 2nd order differential equation, which is accomplished by creating an auxillary
    # variable u = [x', x] that allows us to convert the system into 1st order ode and the variable val stores [x'', x']. 
    # Finally the required function is formed and returned.
    def f_ode(t,u):
        val = np.zeros(2)
        val[0] = f*np.cos(omega*t)-gamma*u[0] - k*u[1]
        val[1] = u[0]
        return val

    
    # Initial & final values of the time and its step values are defined.
    t0 = 0    
    t1 = 20    
    dt = 0.01    
    
    # Number of steps 
    Nt = np.int32(t1/dt)   
    
    # Creating an array of length number of time steps
    t = np.zeros(Nt)    
    
    # Initial conditions
    x_init = [x1,x0]   
    
    # dopri5 represents a higher order Runge-Kutta method
    r = ode(f_ode).set_integrator('dopri5')    
    r.set_initial_value([x0,x1],t0)    
    
    # Time integration
    i = 0
    x = np.zeros(Nt)
    
    # Integrating at every time step
    while r.successful() and r.t < t1:
        r.integrate(r.t + dt)    # integrate on time-interval
        
        # store time point
        t[i] = r.t    
        
        # store solution
        x[i] = r.y[1]    
        i += 1 
        
    # return output quantity   
    return x[Nt-1]   
#print Q([0.1,0.035,0.1,1,0.5,0])

In [4]:
def HarOci(gamma_0,Uncer1,k_0,Uncer2,f_0,Uncer3,omega_0,Uncer4,x0_0,Uncer5,x1_0,Uncer6,var,N):
  np.random.seed(32)    # set random number generator to a fixed value
  Qi = np.zeros(N)    # stores solution at samples
  for i in range(0,N):    # loop over all samples
     z = np.random.rand(6)    # 6 RVs sampled from the uniform distribution in [0,1]
     gamma = ((1-(var/100))+((var*2*z[0])/100))*gamma_0  if Uncer1 else gamma_0  # RVs with 10% variation around nominal value
     omega = ((1-(var/100))+((var*2*z[1])/100))*omega_0  if Uncer2 else omega_0  
     f = ((1-(var/100))+((var*2*z[2])/100))*f_0 if Uncer3 else f_0
     k = ((1-(var/100))+((var*2*z[3])/100))*k_0 if Uncer4 else k_0
     x0 = ((1-(var/100))+((var*2*z[4])/100))*x0_0 if Uncer5 else x0_0
     x1 = ((1-(var/100))+((var*2*z[5])/100))*x1_0 if Uncer6 else x1_0
     Qi[i] = Q([gamma,omega,f,k,x0,x1])
  
  plt.figure(figsize=(10,8))
  plt.xlabel('Values of the output samples')
  plt.ylabel('Frequency of occurrence')  
  plt.hist(Qi,bins=20,histtype='bar',ec='black') 
# Compute statistics and estimated number of samples        
  Q_mean = np.mean(Qi)
  Q_sig = np.std(Qi)
  Q_skew = skew(Qi)
  Q_kurt = kurtosis(Qi)
  print ("The mean estimated by MC method is %g" % Q_mean)
  print ("The standard deviation estimated by MC method is %g" % Q_sig)
  print ("The skewness of the distribution estimated by MC method is %g" % Q_skew)
  print ("The kurtosis of the distribution estimated by MC method is %g" % Q_kurt)
  plt.show() 

In [6]:
def Cant_Beam(Length,Uncer1,Width,Uncer2,Load,Uncer3,Material,Uncer4,var,N):
   np.random.seed(32)
   Y=np.zeros(N)
   for i in range(0,N):    # loop over all samples
     z = np.random.rand(4)
     L = ((1-(var/100))+((var*2*z[0])/100))*Length if Uncer1 else Length     # 6 RVs sampled from the uniform distribution in [0,1]
     W = ((1-(var/100))+((var*2*z[1])/100))*Width if Uncer2 else Width   # RVs with 10% variation around nominal value
     P = ((1-(var/100))+((var*2*z[2])/100))*Load if Uncer3 else Load  
     E = ((1-(var/100))+((var*2*z[3])/100))*Material if Uncer4 else Material
     I =  (W**4)/12
     Y[i]=(P*(L**3))/(3*E*I)
     
   plt.figure(figsize=(10,8))
   plt.xlabel('Values of the output samples')
   plt.ylabel('Frequency of occurrence')  
   plt.hist(Y,bins=20,histtype='bar',ec='black') 
    
  # Compute statistics and estimated number of samples        
   Q_mean = np.mean(Y)
   Q_sig = np.std(Y)
   Q_skew = skew(Y)
   Q_kurt = kurtosis(Y)
   print ("The mean estimated by MC method is %g" % Q_mean)
   print ("The standard deviation estimated by MC method is %g" % Q_sig)
   print ("The skewness of the distribution estimated by MC method is %g" % Q_skew)
   print ("The kurtosis of the distribution estimated by MC method is %g" % Q_kurt)
   plt.show()              

In [23]:
def Skewness(a,b):
   x = np.linspace(skewnorm.ppf(0.01, a),skewnorm.ppf(0.99, a), 100)
   xx = np.linspace(skewnorm.ppf(0.01, b),skewnorm.ppf(0.99, b), 100)
   plt.figure(figsize=(12,8))
   plt.plot(x, skewnorm.pdf(x, a),'r-', lw=2, label='skewed pdf')
   plt.plot(xx, skewnorm.pdf(xx, b),'b-', lw=2, label='standard pdf')
   plt.legend(fontsize = 'medium')
   plt.show()

In [24]:
def KurtPlot(mu,sigma):
  #mu = [0,0,0]
  #sigma = [0.1,0.05,0.03]
  plt.figure(figsize = [12,8])
  def plotting(mu,sigma):
    
    bins = np.linspace(-0.5, 0.5, 1000)
    
    #pdf of a gaussian distribution
    yy = 1/(sigma * np.sqrt(2 * np.pi)) *np.exp( - (bins - mu)**2 / (2 * sigma**2) ) 
    
    # Measuring the kurtosis of the pdf
    kurt = kurtosis(yy) 

    plt.plot(bins,yy,linewidth=2,label='kurtosis=%g'%kurt)
    
  plotting(mu[0],sigma[0])
  plotting(mu[1],sigma[1])
  plotting(mu[2],sigma[2])
  plt.legend(fontsize = 'medium')
  plt.show()